## 2RPFS Problem (Cmax objective) - Tables and Graphs

Before running this, notebook, please run notebooks 0.1 and 0.2 (in this order).

In [ ]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns

%matplotlib inline

In [ ]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

### List files in the result folder 

In [ ]:
resultfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
rpfs_file = os.path.join(resultfolder, '2RPFS_Cmax_all_results.csv')
det_file = os.path.join(resultfolder, 'PFSP_Cmax_deterministic_all_results.csv')

### Create the output folder 

In [ ]:
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
if not os.path.exists(outputfolder):
    os.makedirs(outputfolder)
print('Saving files on folder: ' + outputfolder)

### Process consolidated CSV result files

In [ ]:
df_rpfs = pd.read_csv(rpfs_file, delimiter=';')
df_dpfs = pd.read_csv(det_file, delimiter=';')
df_rpfs.drop(columns=['executionId'], inplace=True)
df_dpfs.drop(columns=['executionId'], inplace=True)

In [ ]:
df_rpfs['time_limit'] = 7200.0
df_rpfs['time'] = np.minimum(df_rpfs['time_spent'], df_rpfs['time_limit'])
df_rpfs['gap'] = df_rpfs['gap'] * 100
df_rpfs['worstcase_cost'] = df_rpfs['cmax_dp']

In [ ]:
df_rpfs_wagner = df_rpfs[(df_rpfs['model'] == 'Wagner')]
df_rpfs_wilson = df_rpfs[(df_rpfs['model'] == 'Wilson')]

In [ ]:
df_rpfs[['time', 'time_spent']]

In [ ]:
df_dpfs.info()

Now, lets join the `rpfs` dataframe with itself (inner join). This will be useful to compare Wilson and Wagner models.

In [ ]:
join_columns = ['n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2']
df_joined = pd.merge(df_rpfs, df_rpfs, how='inner', on=join_columns)
df_joined = df_joined[(df_joined['model_x'] != df_joined['model_y'])]

In [ ]:
df_joined

# Tables

## Table 1. Average worst-case Cmax Wagner 

In [ ]:
table = pd.pivot_table(df_rpfs_wagner, values='cmax_dp', index=['Gamma1', 'Gamma2'], columns=['alpha', 'n'], aggfunc='mean', fill_value=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(table)

## Table 2. Average run time Wagner robust model

In [ ]:
table = pd.pivot_table(df_rpfs_wagner, values='time', index=['Gamma1', 'Gamma2'], columns=['alpha', 'n'], aggfunc='mean', fill_value=0)
table = np.round(table, 2)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(table)

## Table 3. Performance all instances 

In [ ]:
df_joined['x_wins_y_time'] = (df_joined['time_x'] < df_joined['time_y']).astype(int)

In [ ]:
table = pd.pivot_table(df_joined, columns=['n', 'model_x'], values=['time_x', 'gap_x', 'iterations_x', 'x_wins_y_time'],
                       aggfunc={'time_x' : ['mean', 'std'], 'gap_x' : 'mean', 'iterations_x' : ['mean', 'std'],
                               'x_wins_y_time' : ['sum']})  # , margins=True, fill_value=0)
table['perc_x_wins_y_time'] = table['x_wins_y_time'] * 100 / 1250
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(table)

In [ ]:
df_grouped = df.groupby(['alpha', 'n', 'm', 'budget_Gamma']).agg({'executionId' : ['count']}).reset_index()
df_grouped.columns = [ ' '.join(str(i) for i in col) for col in df_grouped.columns]
#df_grouped.reset_index(inplace=True)
df_grouped

In [ ]:
table = pd.pivot_table(df, values='executionId', index=['alpha', 'n'], columns=['Gamma1', 'Gamma2'], aggfunc='count', fill_value=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(table)

# Graphs

### Build a dataframe joining the `df_rpfs` and `df_dpfs`

In [ ]:
join_columns = ['n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2']
df_det_d0 = df_dpfs[(df_dpfs['perc_deviation_p_bar'] == 0)]
df_det_d100 = df_dpfs[(df_dpfs['perc_deviation_p_bar'] == 100)]
# join robust and deterministic dfs
df_join_rob_det = pd.merge(df_rpfs, df_det_d0, how='inner', on=join_columns, suffixes=('_rob', '_d0'))
df_join_rob_det = pd.merge(df_rob_det, df_det_d100, how='inner', on=join_columns, suffixes=('_d0', '_d100'))

### Build a dataframe concatenating `df_rpfs` and `df_dpfs`

In [ ]:
def plot_worstcase_comparison(instance_name, df_dict):
    concat_columns = ['instance_name', 'Gamma1', 'Gamma2', 'budget_Gamma', 'worstcase_cost']
    for key, df_i in df_dict.items():
        df_i = df_i[concat_columns]
        df_i['Method'] = key
        df_dict[key] = df_i
    df = pd.concat(df_dict.values())
    df = df[(df['instance_name'] == instance_name)]
    # https://www.drawingfromdata.com/setting-figure-size-using-seaborn-and-matplotlib
    #fig, ax = plt.subplots()
    # the size of A4 paper
    #fig.set_size_inches(11.7, 8.27)
    marker = ['o', 'x', '^', '+', '*', '8', 's', 'p', 'D', 'V']
    markers = [marker[i] for i in range(len(df["Method"].unique()))]
    linestyle = ['--', '-.', ':', ' ', '', 'solid', 'dashed', 'dashdot', 'dotted', '-']
    linestyles = [linestyle[i] for i in range(len(df["Method"].unique()))]
    a4_dims = (11.7, 8.27)
    plt.figure(figsize=a4_dims)
    sns.catplot(x="budget_Gamma", y="worstcase_cost",  markers=markers, linestyles=linestyles,
                 hue="Method", kind="point", style="Method", 
                 data=df,
                 height=5, # make the plot 5 units high
                 aspect=3) # height should be three times width
    plt.show()
    plt.savefig(os.path.join(outputfolder, '{}.svg'.format(instance_name)))

### Worstcase cost : Small Uncertainty Range Instance - Example 

Alpha = 10% and n = 50

In [ ]:
filename = 'RB0501003.txt'  # Small Uncertainty range
#filename = 'RB0201007.txt'
#filename = 'RB0105010.txt'  # Large Uncertainty range
plot_worstcase_comparison(filename, {'Det_d0' : df_det_d0, 'Det_d100' : df_det_d100, 'Rob' : df_rpfs_wagner})

### Worstcase cost : Large Uncertainty Range Instance - Example 

Alpha = 50% and n = 10

In [ ]:
filename = 'RB0105010.txt'  # Large Uncertainty range
plot_worstcase_comparison(filename, {'Det_d0' : df_det_d0, 'Det_d100' : df_det_d100, 'Rob' : df_rpfs_wagner})

### Export the dataset to CSV file 

In [ ]:
%%time

print('Saving file on folder: ' + rootfolder)
fname = os.path.join(rootfolder, '2RPFS_Cmax_all_results.csv')
df.to_csv(fname, sep=';')
print('Saved: ' + fname)